# Fire from forest

- Authors:
  - Axel Suárez Polo (@ggzor)
  - Sergio Daniel Cortez Chaves (@SerCor)

This notebook shows the training of a neural network to identify fire and smoke within sub-regions
of an image. Most of this notebook shows the process of adjusting the hyperparameters using an automatic
heuristic search (based on average precision and standard deviation).

The dataset was generated by @SerCor, using [tk-tagger](https://github.com/ggzor/tk-tagger), a 
tkinter python application to interactively tag sub-regions of an image; developed to ease the
dataset creation.

[scikit-learn](https://scikit-learn.org/stable/) is used to train the neural network, but it is used
behind a parallel multiprocessing interface with shared memory developed to speed up the training process. 
The relevant module is `experimenter.parallel`.

This notebook goes step-by-step:
  1. Configure logging and some global variables.
  2. Load the dataset using [pandas](https://pandas.pydata.org/).
  3. Configure the experiment ranges.
  4. Do hyperparameter search:
       - Optimal number of neurons and layers.
       - Optimal number of epochs
       - Optimal value for the learning rate.
       - Optimal value for the momentum.

Each step of the hyperparameters search takes the previous `BEST_FOREACH_PHASE` values (default 10) 
from the previous phase and searchs again in the specified range in the configuration.

A heatmap is shown for each experiment, showing the relation of the optimized parameter and the mean 
validation score.

Some graphics of this notebooks are not shown in the default GitHub notebook viewer, because this
notebook makes use of the [altair](https://github.com/altair-viz/altair) visualization library. The
recommended way to view this notebook is using [Visual Studio Code](https://code.visualstudio.com/) which
integrates a [vega](https://github.com/vega/vega) renderer, required by the `altair` library.


## General Configuration

In [28]:
# Enable some logging capabilities
import logging
logging.basicConfig(level='INFO')

# Disable not useful warnings
import os

os.environ["PYTHONWARNINGS"] = "ignore"

# The random seed to use for all the training initializations.
RANDOM_SEED = 0
# The number of subprocesses to use for the parallel training.
POOL_SIZE = 30
# Set to true if you want to see a test run with an approximate duration of 5 minutes.
# Setting this value to `False` is not recommended because it can take hours.
TEST_ENVIRONMENT = True

print(f"Using {POOL_SIZE=}")


Using POOL_SIZE=30


## Data load

The data is loaded with pandas, we load all the `(train, test)` pairs from the `dataset/` folder.

In [29]:
from pathlib import Path
import pandas as pd
import re

# Get all the integer values from the dataset/ files.
filenames_integers = (
    int(m[0]) for p in Path("dataset/").iterdir() if (m := re.search("\d+", str(p)))
)

# The number of partitions to use for the cross validation
PARTITIONS = max(filenames_integers) + 1

datasets = [
    [pd.read_csv(f"dataset/{ftype}_{i}.csv") for ftype in ["train", "test"]]
    for i in range(PARTITIONS)
]

datasets[0][0]


,rgb__mean_c0,rgb__mean_c1,rgb__mean_c2,rgb__stdev_c0,rgb__stdev_c1,rgb__stdev_c2,rgb__median_c0,rgb__median_c1,rgb__median_c2,rgb__cov_0,...,hsv__median_c0,hsv__median_c1,hsv__median_c2,hsv__cov_0,hsv__cov_1,hsv__cov_2,gray__mean,gray__stdev,gray__median,tag
0,-0.854147,-0.833795,-0.853242,-0.491280,-0.470497,-0.446814,-0.857708,-0.861224,-0.898734,-0.797766,...,-0.763448,-0.130435,-0.857708,-0.271764,0.263642,-0.137542,-0.836924,-0.431625,-0.873950,OTHER
1,-0.343325,-0.108892,-0.585593,-0.234473,-0.033132,-0.202226,-0.343874,-0.134694,-0.662447,-0.543264,...,-0.554119,0.209490,-0.162055,-0.206723,0.207260,-0.324056,-0.176654,-0.044259,-0.210084,OTHER
2,-0.183933,0.237009,-0.409963,-0.249102,-0.181247,-0.203540,-0.154150,0.208163,-0.417722,-0.589167,...,-0.488982,0.058824,0.154150,-0.195829,0.157020,-0.275458,0.122746,-0.146074,0.109244,OTHER
3,-0.680830,-0.633464,-0.654085,-0.186207,-0.153143,-0.158475,-0.731225,-0.722449,-0.755274,-0.519254,...,-0.662069,-0.517241,-0.723320,-0.216011,0.194512,-0.189558,-0.638210,-0.098288,-0.731092,OTHER
4,-0.171050,0.088423,-0.500224,-0.081952,-0.040828,-0.226752,-0.083004,0.110204,-0.468354,-0.448166,...,-0.558090,0.119334,0.098814,-0.143581,0.115447,-0.335477,0.011585,-0.014120,0.050420,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,-0.182917,-0.102501,-0.018490,-0.866989,-0.857741,-0.844239,-0.169960,-0.134694,-0.037975,-0.968427,...,0.393966,-0.858407,-0.114625,-0.195889,0.149306,0.003867,-0.092206,-0.842001,-0.117647,SMOKE
1643,0.317514,-0.588199,-0.997015,-0.887188,-0.997200,-0.990703,0.335968,-0.583673,-1.000000,-0.985231,...,-0.900609,1.000000,0.335968,-0.196159,0.150517,0.004518,-0.402942,-0.982982,-0.411765,SMOKE
1644,0.034336,0.121826,0.175029,-0.028794,0.069586,0.016202,0.146245,0.216327,0.291139,-0.288142,...,0.259561,-0.817204,0.252964,-0.167841,-0.187197,-0.037730,0.132040,0.121805,0.243697,SMOKE
1645,0.247614,-0.348639,-0.847863,-0.923813,-0.954711,-0.960968,0.256917,-0.363265,-0.848101,-0.980592,...,-0.856562,0.775148,0.256917,-0.196021,0.151759,0.005965,-0.235698,-0.942772,-0.252101,SMOKE


A label encoder is used to transform the tags of the data frames to numbers.

In [30]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(
    [tag for experiment in datasets for df in experiment for tag in df["tag"].unique()]
)

label_encoder.classes_


array(['FIRE', 'OTHER', 'SMOKE'], dtype='<U5')

A `DataExperiment` is created for each `(train, test)` dataframe pair.

In [31]:
from experimenter.model import DataExperiment


def to_data_experiment(train_test_df) -> DataExperiment:
    train, test = train_test_df

    return DataExperiment(
        train.loc[:, train.columns != "tag"].to_numpy(),
        label_encoder.transform(train["tag"]),
        test.loc[:, train.columns != "tag"].to_numpy(),
        label_encoder.transform(test["tag"]),
    )


all_experiments = [to_data_experiment(pair) for pair in datasets]


## Experiment configuration

In [32]:
from pprint import pprint
from functools import reduce
import numpy as np

# The number of classes within the dataset
CLASSES = label_encoder.classes_.size
# The number of attributes found in the dataset
ATTRIBUTES = datasets[0][0].columns.size - 1

# Number of best parameters to keep for each training phase
BEST_FOREACH_PHASE= 10

# A range for each optimizable hyperparameter
NEURONS_RANGE = np.arange(CLASSES, ATTRIBUTES + CLASSES + 1, 1)
LAYERS_RANGE = np.arange(1, 7 + 1, 1)
EPOCHS_RANGE = np.arange(50, 300 + 1, 10)
LEARNING_RATE_RANGE = np.around(np.linspace(0.001, 0.200, 30), 4)
MOMENTUM_RANGE = np.around(np.linspace(0.01, 0.4, 30), 4)

# Get the complete total number of experiments
# Note that this is just the theoretical number of experiments that had
# to be run if the best hyperparameters were to be found
total_experiments = reduce(
    lambda x, y: x * y,
    map(
        lambda arr: np.size(arr, 0),
        [
            NEURONS_RANGE,
            LAYERS_RANGE,
            EPOCHS_RANGE,
            LEARNING_RATE_RANGE,
            MOMENTUM_RANGE
        ],
    ),
)
print(f"Complete total: {total_experiments}")

Complete total: 6552000


## Visual configuration

This variables are used to show the results in the heatmaps within using too much
vertical space.

In [33]:
NEURON_BIN_SIZE = 20
EPOCH_BIN_SIZE = 600
LEARNING_RATE_BIN_SIZE = 0.300
MOMENTUM_BIN_SIZE = 0.500

# Sometimes the accuracy is lower than this number, that makes some
# details disappear, so this treshold is used to show more details.
DETAILED_VIEW_TRESHOLD = 85

## Experimentation

A `ParallelExperimenter` is created with the `DataExperiment`'s that were created in the previous step.
This class will allow us to run our training in parallel.

In [34]:
from experimenter.parallel import ParallelExperimenter
experimenter = ParallelExperimenter(all_experiments, POOL_SIZE, RANDOM_SEED)


## Test parameters

If the `TEST_ENVIRONMENT` flag is set, just a fraction of the hyperparameter tuning phases will be ran.

In [35]:
if TEST_ENVIRONMENT:
    NEURONS_RANGE = NEURONS_RANGE[: NEURONS_RANGE.size // 4]
    LAYERS_RANGE = LAYERS_RANGE[: NEURONS_RANGE.size // 4]
    EPOCHS_RANGE = EPOCHS_RANGE[: EPOCHS_RANGE.size // 5]
    LEARNING_RATE_RANGE = LEARNING_RATE_RANGE[: LEARNING_RATE_RANGE.size // 4]
    MOMENTUM_RANGE = MOMENTUM_RANGE[: MOMENTUM_RANGE.size // 4]


## Initial experimentation

The initial experimentation values are set:

In [36]:
INITIAL_NEURONS = NEURONS_RANGE[0]
INITIAL_LAYERS = LAYERS_RANGE[0]
INITIAL_EPOCHS = EPOCHS_RANGE[EPOCHS_RANGE.size // 4]
INITIAL_LEARNING_RATE = LEARNING_RATE_RANGE[LEARNING_RATE_RANGE.size // 2]
INITIAL_MOMENTUM = MOMENTUM_RANGE[MOMENTUM_RANGE.size // 2]

pprint(
    {k: v for k, v in locals().items() if k.isupper() and k.startswith("INITIAL")},
    sort_dicts=False,
)

# This is the real number of experiments that will be ran from the entire total shown above.
real_experiments = NEURONS_RANGE.size * LAYERS_RANGE.size + BEST_FOREACH_PHASE * (
    EPOCHS_RANGE.size + LEARNING_RATE_RANGE.size + MOMENTUM_RANGE.size
)
print(f"Real experiments count: {real_experiments}")


{'INITIAL_NEURONS': 3,
 'INITIAL_LAYERS': 1,
 'INITIAL_EPOCHS': 60,
 'INITIAL_LEARNING_RATE': 0.0216,
 'INITIAL_MOMENTUM': 0.0503}
Real experiments count: 210


## Hyperparameter tuning phase

Now the hyperparameters are tuned. The first step is to find the best neurons and layers parameters, where
"better" is when the accuracy is higher and the standard deviation is low.

### Neurons and layers tuning

In [37]:
from experimenter.model import ModelParams
import dataclasses

initial_experiment = ModelParams(
    INITIAL_NEURONS,
    layers=INITIAL_LAYERS,
    epochs=INITIAL_EPOCHS,
    learning_rate=INITIAL_LEARNING_RATE,
    momentum=INITIAL_MOMENTUM,
)

print(f'Running {NEURONS_RANGE.size * LAYERS_RANGE.size} experiments')
neurons_result = experimenter.run_all(
    [
        dataclasses.replace(initial_experiment, neurons=neurons, layers=layers)
        for neurons in NEURONS_RANGE
        for layers in LAYERS_RANGE
    ]
)

Running 20 experiments


INFO:experimenter.parallel:0 / 20
INFO:experimenter.parallel:1 / 20
INFO:experimenter.parallel:2 / 20
INFO:experimenter.parallel:3 / 20
INFO:experimenter.parallel:4 / 20
INFO:experimenter.parallel:5 / 20
INFO:experimenter.parallel:6 / 20
INFO:experimenter.parallel:7 / 20
INFO:experimenter.parallel:8 / 20
INFO:experimenter.parallel:9 / 20
INFO:experimenter.parallel:10 / 20
INFO:experimenter.parallel:11 / 20
INFO:experimenter.parallel:12 / 20
INFO:experimenter.parallel:13 / 20
INFO:experimenter.parallel:14 / 20
INFO:experimenter.parallel:15 / 20
INFO:experimenter.parallel:16 / 20
INFO:experimenter.parallel:17 / 20
INFO:experimenter.parallel:18 / 20
INFO:experimenter.parallel:19 / 20
INFO:experimenter.parallel:20 / 20


In [38]:
from experimenter.model import ExperimentResult
from typing import List

TOLERABLE_DEVIATION = 0.4
NEGATIVE_DEVIATION_WEIGHT = -10


def experiment_result_criteria(experiment: ExperimentResult) -> float:
    deviation_factor = 0
    if experiment.stddev > TOLERABLE_DEVIATION:
        deviation_factor = NEGATIVE_DEVIATION_WEIGHT * experiment.stddev

    return experiment.mean + deviation_factor


def to_dataframe(experiments: List[ExperimentResult]):
    return pd.DataFrame(
        [
            {
                **dataclasses.asdict(result.params),
                "result_mean": result.mean,
                "result_std": result.stddev,
            }
            for result in experiments
        ]
    )


neurons_result.sort(key=experiment_result_criteria, reverse=True)
neurons_result_df = to_dataframe(neurons_result)

neurons_result_df.head(BEST_FOREACH_PHASE)[["result_mean", "result_std", "neurons", "layers"]]


,result_mean,result_std,neurons,layers
0,88.684832,1.126049,5,2
1,89.850235,1.273509,11,2
2,88.879125,1.233917,10,1
3,88.830463,1.230597,8,1
4,89.073300,1.286939,9,2
5,87.324546,1.254745,6,2
6,89.122198,1.476473,11,1
7,88.539201,1.460776,8,2
8,88.976212,1.518471,9,1
9,90.384924,1.659414,12,2


In [39]:
import altair as alt


def heatmap(title, source, tooltip="props", **kwargs):
    if tooltip == "props":
        tooltip = source.columns.to_list()

    return (
        alt.Chart(source)
        .mark_rect()
        .encode(tooltip=tooltip, **kwargs)
        .properties(title=title)
        .interactive()
    )


def binify(dataframe, param, bins):
    return [dataframe[dataframe[param].isin(bin)] for bin in bins]


def binified_heatmap(title, source, param, bins, tooltip="props", **kwargs):
    return alt.hconcat(
        *(
            heatmap(title, source[source[param].isin(bin)], tooltip=tooltip, **kwargs)
            for bin in bins
        )
    )


neurons_result_encoding = {"x": "layers:O", "y": "neurons:O", "color": "result_mean:Q"}
neurons_result_tooltip = ["result_mean", "result_std", "neurons", "layers"]
NEURON_BINS = np.array_split(NEURONS_RANGE, NEURONS_RANGE.max() // NEURON_BIN_SIZE + 1)


def binified_neurons_heatmap(title, neurons_df):
    return binified_heatmap(
        title,
        neurons_df,
        param="neurons",
        bins=NEURON_BINS,
        tooltip=neurons_result_tooltip,
        **neurons_result_encoding
    )


binified_neurons_heatmap("Mean precision", neurons_result_df)


alt.HConcatChart(...)

In [40]:
range_neurons_result_df = neurons_result_df[
    neurons_result_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_neurons_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", range_neurons_result_df
)


alt.HConcatChart(...)

### Epochs tuning

In [42]:
best_by_neurons_layers = list(n.params for n in neurons_result[:BEST_FOREACH_PHASE])
print(f"Running {BEST_FOREACH_PHASE * EPOCHS_RANGE.size} experiments")
epochs_results = experimenter.run_all(
    [
        dataclasses.replace(p, epochs=epochs)
        for p in best_by_neurons_layers
        for epochs in EPOCHS_RANGE
    ]
)


Running 50 experiments


INFO:experimenter.parallel:0 / 50
INFO:experimenter.parallel:1 / 50
INFO:experimenter.parallel:2 / 50
INFO:experimenter.parallel:3 / 50
INFO:experimenter.parallel:4 / 50
INFO:experimenter.parallel:5 / 50
INFO:experimenter.parallel:6 / 50
INFO:experimenter.parallel:7 / 50
INFO:experimenter.parallel:8 / 50
INFO:experimenter.parallel:9 / 50
INFO:experimenter.parallel:10 / 50
INFO:experimenter.parallel:11 / 50
INFO:experimenter.parallel:12 / 50
INFO:experimenter.parallel:13 / 50
INFO:experimenter.parallel:14 / 50
INFO:experimenter.parallel:15 / 50
INFO:experimenter.parallel:16 / 50
INFO:experimenter.parallel:17 / 50
INFO:experimenter.parallel:18 / 50
INFO:experimenter.parallel:19 / 50
INFO:experimenter.parallel:20 / 50
INFO:experimenter.parallel:21 / 50
INFO:experimenter.parallel:22 / 50
INFO:experimenter.parallel:23 / 50
INFO:experimenter.parallel:24 / 50
INFO:experimenter.parallel:25 / 50
INFO:experimenter.parallel:26 / 50
INFO:experimenter.parallel:27 / 50
INFO:experimenter.parallel:28 

In [43]:
epochs_results.sort(key=experiment_result_criteria, reverse=True)
epochs_results_reverse_index = {
    (r.neurons, r.layers): i for i, r in enumerate(best_by_neurons_layers)
}

epochs_results_df = to_dataframe(epochs_results)

epochs_results_df["src_index"] = epochs_results_df.apply(
    lambda row: epochs_results_reverse_index[row["neurons"], row["layers"]], axis=1
)

epochs_results_df.head(BEST_FOREACH_PHASE)[
    ["result_mean", "result_std", "neurons", "layers", "epochs", "src_index"]
]


,result_mean,result_std,neurons,layers,epochs,src_index
0,88.927314,1.053060,6,2,90,5
1,89.510311,1.115158,8,1,70,3
2,89.073182,1.078459,10,1,70,2
3,90.627052,1.239740,11,2,70,1
4,89.753148,1.166966,9,2,90,4
5,90.627170,1.267287,11,2,80,1
6,88.684832,1.126049,5,2,60,0
7,88.781683,1.146704,6,2,80,5
8,89.850235,1.273509,11,2,60,1
9,89.413342,1.236393,11,1,80,6


In [44]:
epochs_results_encoding = {
    "x": "src_index:O",
    "y": "epochs:O",
    "color": "result_mean:Q",
}
epochs_results_tooltip = ["result_mean", "result_std", "neurons", "layers", "epochs", "src_index"]
EPOCHS_BINS = np.array_split(EPOCHS_RANGE, EPOCHS_RANGE.max() // EPOCH_BIN_SIZE + 1)


def binified_epochs_heatmap(title, epochs_df):
    return binified_heatmap(
        title,
        epochs_df,
        "epochs",
        EPOCHS_BINS,
        epochs_results_tooltip,
        **epochs_results_encoding
    )


binified_epochs_heatmap("Mean precision", epochs_results_df)


alt.HConcatChart(...)

In [45]:
range_epoch_results_df = epochs_results_df[
    epochs_results_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_epochs_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", 
    range_epoch_results_df
)


alt.HConcatChart(...)

### Learning rate tuning

In [47]:
best_by_epochs = list(r.params for r in epochs_results[:BEST_FOREACH_PHASE])

print(f"Running {BEST_FOREACH_PHASE * LEARNING_RATE_RANGE.size} experiments")

learning_rate_results = experimenter.run_all(
    [
        dataclasses.replace(p, learning_rate=learning_rate)
        for p in best_by_epochs
        for learning_rate in LEARNING_RATE_RANGE
    ]
)


Running 70 experiments


INFO:experimenter.parallel:0 / 70
INFO:experimenter.parallel:1 / 70
INFO:experimenter.parallel:2 / 70
INFO:experimenter.parallel:3 / 70
INFO:experimenter.parallel:4 / 70
INFO:experimenter.parallel:5 / 70
INFO:experimenter.parallel:6 / 70
INFO:experimenter.parallel:7 / 70
INFO:experimenter.parallel:8 / 70
INFO:experimenter.parallel:9 / 70
INFO:experimenter.parallel:10 / 70
INFO:experimenter.parallel:11 / 70
INFO:experimenter.parallel:12 / 70
INFO:experimenter.parallel:13 / 70
INFO:experimenter.parallel:14 / 70
INFO:experimenter.parallel:15 / 70
INFO:experimenter.parallel:16 / 70
INFO:experimenter.parallel:17 / 70
INFO:experimenter.parallel:18 / 70
INFO:experimenter.parallel:19 / 70
INFO:experimenter.parallel:20 / 70
INFO:experimenter.parallel:21 / 70
INFO:experimenter.parallel:22 / 70
INFO:experimenter.parallel:23 / 70
INFO:experimenter.parallel:24 / 70
INFO:experimenter.parallel:25 / 70
INFO:experimenter.parallel:26 / 70
INFO:experimenter.parallel:27 / 70
INFO:experimenter.parallel:28 

In [48]:
learning_rate_results.sort(key=experiment_result_criteria, reverse=True)
learning_rate_results_reverse_index = {
    (r.neurons, r.layers, r.epochs): i for i, r in enumerate(best_by_epochs)
}

learning_rate_results_df = to_dataframe(learning_rate_results)

learning_rate_results_df["src_index"] = learning_rate_results_df.apply(
    lambda row: learning_rate_results_reverse_index[
        row["neurons"], row["layers"], row["epochs"]
    ],
    axis=1,
)

learning_rate_results_df.head(BEST_FOREACH_PHASE)[
    [
        "result_mean",
        "result_std",
        "neurons",
        "layers",
        "epochs",
        "learning_rate",
        "src_index",
    ]
]


,result_mean,result_std,neurons,layers,epochs,learning_rate,src_index
0,91.647060,1.313562,11,2,80,0.0422,5
1,88.927314,1.053060,6,2,90,0.0216,0
2,89.510311,1.115158,8,1,70,0.0216,1
3,89.073182,1.078459,10,1,70,0.0216,2
4,90.627052,1.239740,11,2,70,0.0216,3
5,90.044528,1.193263,10,1,70,0.0353,2
6,89.753148,1.166966,9,2,90,0.0216,4
7,89.607517,1.156305,11,2,80,0.0147,5
8,90.627170,1.267287,11,2,80,0.0216,5
9,91.161269,1.322848,11,2,70,0.0422,3


In [49]:
learning_rate_results_encoding = {
    "x": "src_index:O",
    "y": "learning_rate:O",
    "color": "result_mean:Q",
}
learning_rate_results_tooltip = [
    "result_mean",
    "result_std",
    "neurons",
    "layers",
    "epochs",
    "learning_rate",
    "src_index",
]
LEARNING_RATE_BINS = np.array_split(
    LEARNING_RATE_RANGE, LEARNING_RATE_RANGE.max() // LEARNING_RATE_BIN_SIZE + 1
)


def binified_learning_rate_heatmap(title, lr_df):
    return binified_heatmap(
        title,
        lr_df,
        "learning_rate",
        LEARNING_RATE_BINS,
        learning_rate_results_tooltip,
        **learning_rate_results_encoding
    )


binified_learning_rate_heatmap("Mean precision", learning_rate_results_df)


alt.HConcatChart(...)

In [50]:
range_learning_rate_results_df = learning_rate_results_df[
    learning_rate_results_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_learning_rate_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", range_learning_rate_results_df
)


alt.HConcatChart(...)

### Momentum tuning

In [52]:
best_by_learning_rate = list(
    r.params for r in learning_rate_results[:BEST_FOREACH_PHASE]
)

print(f"Running {BEST_FOREACH_PHASE * MOMENTUM_RANGE.size} experiments")

momentum_results = experimenter.run_all(
    [
        dataclasses.replace(p, momentum=momentum)
        for p in best_by_learning_rate
        for momentum in MOMENTUM_RANGE
    ]
)


Running 70 experiments


INFO:experimenter.parallel:0 / 70
INFO:experimenter.parallel:1 / 70
INFO:experimenter.parallel:2 / 70
INFO:experimenter.parallel:3 / 70
INFO:experimenter.parallel:4 / 70
INFO:experimenter.parallel:5 / 70
INFO:experimenter.parallel:6 / 70
INFO:experimenter.parallel:7 / 70
INFO:experimenter.parallel:8 / 70
INFO:experimenter.parallel:9 / 70
INFO:experimenter.parallel:10 / 70
INFO:experimenter.parallel:11 / 70
INFO:experimenter.parallel:12 / 70
INFO:experimenter.parallel:13 / 70
INFO:experimenter.parallel:14 / 70
INFO:experimenter.parallel:15 / 70
INFO:experimenter.parallel:16 / 70
INFO:experimenter.parallel:17 / 70
INFO:experimenter.parallel:18 / 70
INFO:experimenter.parallel:19 / 70
INFO:experimenter.parallel:20 / 70
INFO:experimenter.parallel:21 / 70
INFO:experimenter.parallel:22 / 70
INFO:experimenter.parallel:23 / 70
INFO:experimenter.parallel:24 / 70
INFO:experimenter.parallel:25 / 70
INFO:experimenter.parallel:26 / 70
INFO:experimenter.parallel:27 / 70
INFO:experimenter.parallel:28 

## Best results

The best results found are show below.

In [53]:
momentum_results.sort(key=experiment_result_criteria, reverse=True)
momentum_results_reverse_index = {
    (r.neurons, r.layers, r.epochs, r.learning_rate): i for i, r in enumerate(best_by_learning_rate)
}

momentum_results_df = to_dataframe(momentum_results)

momentum_results_df["src_index"] = momentum_results_df.apply(
    lambda row: momentum_results_reverse_index[
        row["neurons"], row["layers"], row["epochs"], row["learning_rate"]
    ],
    axis=1,
)

momentum_results_df.head(BEST_FOREACH_PHASE)[
    [
        "result_mean",
        "result_std",
        "neurons",
        "layers",
        "epochs",
        "learning_rate",
        "momentum",
        "src_index",
    ]
]

,result_mean,result_std,neurons,layers,epochs,learning_rate,momentum,src_index
0,91.598162,1.142583,11,2,80,0.0422,0.0907,0
1,89.655942,1.006916,11,2,80,0.0147,0.0638,7
2,89.704486,1.018752,11,2,80,0.0147,0.0772,7
3,91.744148,1.253228,11,2,80,0.0422,0.0638,0
4,91.744148,1.253228,11,2,80,0.0422,0.0772,0
5,91.695604,1.284667,11,2,80,0.0422,0.0369,0
6,90.093072,1.126383,10,1,70,0.0353,0.0369,5
7,90.530083,1.174324,11,2,80,0.0216,0.0234,8
8,89.850235,1.115701,11,2,80,0.0147,0.0907,7
9,88.878771,1.027927,6,2,90,0.0216,0.0772,1


In [54]:
momentum_results_encoding = {
    "x": "src_index:O",
    "y": "momentum:O",
    "color": "result_mean:Q",
}
momentum_results_tooltip = [
    "result_mean",
    "result_std",
    "neurons",
    "layers",
    "epochs",
    "learning_rate",
    "momentum",
    "src_index",
]
MOMENTUM_BINS = np.array_split(
    MOMENTUM_RANGE, MOMENTUM_RANGE.max() // MOMENTUM_BIN_SIZE + 1
)


def binified_momentum_heatmap(title, momentum_df):
    return binified_heatmap(
        title,
        momentum_df,
        "momentum",
        MOMENTUM_BINS,
        momentum_results_tooltip,
        **momentum_results_encoding
    )


binified_momentum_heatmap("Mean precision", momentum_results_df)


alt.HConcatChart(...)

In [55]:
range_momentum_results_df = momentum_results_df[
    momentum_results_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_momentum_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", range_momentum_results_df
)


alt.HConcatChart(...)